<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

# Neuer Abschnitt

<p><font size="6" color='grey'> <b>
SQL RAG
</b></font> </br></p>


---

<a target="_blank" href="https://colab.research.google.com/github/ralf-42/GenAI/blob/main/01%20ipynb/M00_Prolog.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# **1 <font color='orange'>|</font> Einführung in SAG**
---

SQL-Augmented Generation (SAG) ist eine Technologie, die Large Language Models (LLMs) mit Datenbankabfragen kombiniert. Sie ermöglicht es, natürlichsprachliche Anfragen in SQL-Abfragen zu übersetzen und die Ergebnisse intelligent zu interpretieren.

Diese Technologie überbrückt die Lücke zwischen menschlicher Sprache und Datenbankstrukturen, indem sie:

- Natürliche Sprache in präzise SQL-Abfragen umwandelt
- Datenbankschemas analysiert, um korrekte Abfragen zu generieren
- Die Abfrageergebnisse in verständliche Antworten umformuliert

SAG erweitert die Fähigkeiten von LLMs, indem es ihnen Zugriff auf strukturierte Daten ermöglicht und so präzisere, faktenbasierte Antworten liefert.




# **2 <font color='orange'>|</font> Vergleich zu RAG**
---

Während sowohl SAG als auch RAG (Retrieval-Augmented Generation) die Fähigkeiten von LLMs erweitern, gibt es wichtige Unterschiede:



| Merkmal         | SQL-Augmented Generation (SAG)       | Retrieval-Augmented Generation (RAG)    |
| --------------- | ------------------------------------ | --------------------------------------- |
| Datenquelle     | Strukturierte Datenbanken            | Textdokumente, Wissensbasen             |
| Abfragemethode  | SQL-Generierung                      | Semantische Suche, Embedding-Vergleiche |
| Datenstruktur   | Schema-basiert, tabellarisch         | Unstrukturiert oder semi-strukturiert   |
| Genauigkeit     | Präzise durch Datenbankintegrität    | Abhängig von der Retrieval-Qualität     |
| Anwendungsfälle | Geschäftsanalysen, Berichterstellung | Dokumentensuche, Wissensbasis-Anfragen  |
| Aktualisierung  | In Echtzeit durch aktuelle DB-Daten  | Erfordert Neuindexierung bei Änderungen |

SAG eignet sich besonders für Szenarien, in denen präzise, aktuelle Daten benötigt werden, während RAG Stärken bei der Verarbeitung großer Textmengen hat.



# **3 <font color='orange'>|</font> Integration LLM und DB**
---



Die Integration von LLMs mit Datenbanken erfolgt über mehrere Komponenten:

1. **Schema-Analyse**: Das LLM muss das Datenbankschema verstehen (Tabellen, Spalten, Beziehungen)
2. **Anfrage-Übersetzung**: Umwandlung der natürlichsprachlichen Anfrage in SQL
3. **Abfrage-Ausführung**: Verbindung zur Datenbank und Ausführung der generierten SQL-Abfrage
4. **Ergebnis-Interpretation**: Analyse und Interpretation der Abfrageergebnisse

In [ ]:
!uv pip install --system -q langchain_openai langchain_experimental

In [ ]:
# Northwind-Datenbank herunterladen
!rm -rf northwind.db
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/northwind.db -o northwind.db

In [ ]:
# Grundlegender SAG-Ablauf
from langchain_openai import ChatOpenAI
from langchain_experimental.sql.base import SQLDatabase

# 1. Datenbankverbindung herstellen
db = SQLDatabase.from_uri("sqlite:///northwind.db")

# 2. LLM initialisieren
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# 3. Datenbankschema abrufen
schema = db.get_table_info()
print(schema)

# 4. Natürlichsprachliche Anfrage
user_query = "Wie viele Mitarbeiter haben wir?"

# 5. SQL-Abfrage generieren und ausführen
# (Detaillierte Umsetzung folgt in späteren Abschnitten)

Die Herausforderung liegt in der korrekten Interpretation des Schemas und der präzisen Übersetzung der Anfragen.



# **4 <font color='orange'>|</font> SQL-Generierung mit LLMs**
---



Die SQL-Generierung ist ein kritischer Bestandteil von SAG und erfolgt in mehreren Schritten:

1. **Prompt-Engineering**: Entwicklung spezifischer Prompts, die das Datenbankschema und die Anforderungen enthalten
2. **Query-Planung**: Analyse der Anfrage, um die benötigten Tabellen und Joins zu identifizieren
3. **SQL-Syntax-Generierung**: Erzeugung syntaktisch korrekter SQL-Abfragen
4. **Validierung**: Überprüfung der generierten Abfrage vor der Ausführung

In [ ]:
import re
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# SQL-Generierungs-Prompt
sql_template = """
Du bist ein SQL-Experte. Deine Aufgabe ist es, Benutzeranfragen in SQL-Abfragen zu übersetzen.
Verwende die SQLite-Syntax und nur die Tabellen und Spalten aus dem bereitgestellten Schema.
Schreibe NUR die SQL-Abfrage ohne Präfixe oder Kommentare.

Datenbank-Schema:
{schema}

Benutzeranfrage: {query}

SQL-Abfrage:
"""

# SQL-Generator-Chain
sql_generator = (
    RunnablePassthrough.assign(schema=lambda _: db.get_table_info())
    | PromptTemplate.from_template(sql_template)
    | llm
    | StrOutputParser()
)

# Verwendung
sql_query = sql_generator.invoke({"query": user_query})
# Bereinigung um
sql_query = re.sub(r'```sql\s*(.*?)\s*```', r'\1', sql_query, flags=re.DOTALL)
sql_query = sql_query.replace("```", "").strip()


sql_query

Fortgeschrittene Implementierungen können Techniken wie Few-Shot-Learning und spezifische SQL-Formatvorgaben nutzen, um die Qualität zu verbessern.


# **5 <font color='orange'>|</font> Validierung und Sicherheit**
---



Die Sicherheit ist bei der Arbeit mit datenbankgesteuerten Anwendungen von entscheidender Bedeutung. SAG-Implementierungen müssen folgende Sicherheitsaspekte berücksichtigen:

1. **SQL-Injection-Prävention**:
    
    - Validierung und Bereinigung generierter SQL-Abfragen
    - Verwendung von parametrisierten Abfragen
    - Beschränkung der SQL-Befehle (z.B. nur SELECT-Anweisungen zulassen)
2. **Zugriffskontrolle**:
    
    - Verwendung von Datenbanknutzern mit eingeschränkten Rechten
    - Zugriffsbeschränkungen auf bestimmte Tabellen oder Ansichten
    - Implementierung von Row-Level-Security
3. **Datenvalidierung**:
    
    - Überprüfung der generierten SQL-Abfragen auf verdächtige Muster
    - Begrenzung der Abfragekomplexität und -länge
    - Timeouts für lang laufende Abfragen


In [ ]:
def validate_sql_query(sql_query):
    """Validiert eine SQL-Abfrage auf potenziell gefährliche Muster."""

    # Nur SELECT-Anweisungen erlauben
    if not sql_query.strip().upper().startswith("SELECT"):
        return False, "Nur SELECT-Anweisungen sind erlaubt."

    # Keine gefährlichen SQL-Befehle erlauben
    dangerous_commands = ["DROP", "DELETE", "TRUNCATE", "UPDATE", "INSERT", "ALTER"]
    for command in dangerous_commands:
        if f" {command} " in sql_query.upper():
            return False, f"Unerlaubter SQL-Befehl: {command}"

    # Weitere Validierungsregeln...

    return True, "SQL-Abfrage ist gültig."

# Verwendung
is_valid, message = validate_sql_query(sql_query)
is_valid, message

Eine gründliche Validierung vor der Ausführung ist entscheidend für die Sicherheit der Anwendung.


# **6 <font color='orange'>|</font> Praktische Anwendungsfälle**
---

SAG eignet sich für zahlreiche praktische Anwendungsfälle:

1. **Business Intelligence Dashboards**:
    
    - Natürlichsprachliche Abfragen für Geschäftskennzahlen
    - Dynamische Berichte basierend auf Benutzeranfragen
    - Trends und Anomalien in Daten identifizieren
2. **Datenanalyse für Nicht-Techniker**:
    
    - Ermöglicht Benutzern ohne SQL-Kenntnisse, komplexe Datenabfragen durchzuführen
    - Vereinfacht den Zugang zu Unternehmensdaten
3. **Automatisierte Berichterstellung**:
    
    - Generierung regelmäßiger Berichte basierend auf Datenabfragen
    - Intelligente Zusammenfassung und Interpretation von Geschäftsdaten
4. **Kundenservice-Anwendungen**:
    
    - Schneller Zugriff auf Kundendaten für Support-Mitarbeiter
    - Automatisierte Beantwortung häufiger Kundenanfragen
5. **Interne Wissensmanagement-Systeme**:
    
    - Intelligente Suche in Unternehmensdaten
    - Verknüpfung verschiedener Datenquellen für umfassende Antworten

Durch die Kombination von LLMs mit Datenbankabfragen kann SAG komplexe Analyseaufgaben automatisieren und den Zugang zu Daten demokratisieren.



# **7 <font color='orange'>|</font> SAG mit LangChain**
---



LangChain bietet leistungsstarke Tools für die Implementierung von SAG-Lösungen:

1. **SQLDatabaseChain**: Eine spezialisierte Chain für Datenbankinteraktionen
2. **SQLDatabaseToolkit**: Werkzeuge zur vereinfachten Interaktion mit Datenbanken
3. **Erweiterte Prompt-Templates**: Spezifisch für SQL-Generierung optimierte Prompts

Hier ist ein Beispiel für die Implementierung einer einfachen SAG-Anwendung mit LangChain:

In [ ]:
from langchain_experimental.sql.base import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Datenbankverbindung herstellen
db = SQLDatabase.from_uri("sqlite:///northwind.db")

# LLM initialisieren
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# SQL-Generierungs-Prompt
template = """
Du bist ein SQL-Experte. Übersetze folgende Anfrage in SQL.
Verwende nur die Tabellen und Spalten aus dem bereitgestellten Schema.

Datenbank-Schema:
{schema}

Benutzeranfrage: {query}

SQL-Abfrage:
"""

# Ergebnis-Interpretations-Prompt
result_template = """
Interpretiere die folgenden SQL-Abfrageergebnisse, um die Benutzeranfrage zu beantworten:

Benutzeranfrage: {query}
SQL-Abfrage: {sql_query}
Abfrageergebnisse:
{results}

Deine Analyse:
"""

# SQL-Generator-Chain
sql_generator = (
    RunnablePassthrough.assign(schema=lambda _: db.get_table_info())
    | PromptTemplate.from_template(template)
    | llm
    | StrOutputParser()
)

# Funktion zum Ausführen der Abfrage
def execute_query(sql_query):
    # Implementierung der Abfrageausführung...
    pass

# Ergebnis-Analyse-Chain
result_analyzer = (
    PromptTemplate.from_template(result_template)
    | llm
    | StrOutputParser()
)

# Vollständige SAG-Chain
def process_query(user_query):
    # SQL generieren
    sql_query = sql_generator.invoke({"query": user_query})

    # SQL ausführen
    results = execute_query(sql_query)

    # Ergebnisse analysieren
    analysis = result_analyzer.invoke({
        "query": user_query,
        "sql_query": sql_query,
        "results": results
    })

    return {
        "sql_query": sql_query,
        "results": results,
        "analysis": analysis
    }

# **8 <font color='orange'>|</font> SAG Northwind**
---

Hier ist ein vollständiges Beispiel für eine SAG-Anwendung:

<p><font color='black' size="5">
Erläuterung des SAG-Beispiels
</font></p>

Das Beispiel demonstriert eine vollständige SAG-Anwendung mit folgenden Komponenten:

1. **Datenbankintegration**: Northwind-Datenbank über SQLite
2. **LLM-Anbindung**: Verwendung des ChatOpenAI-Modells von OpenAI
3. **SQL-Generierungs-Chain**: Umwandlung natürlicher Sprache in SQL
4. **Abfrageausführung**: Sichere Ausführung und Formatierung der Ergebnisse
5. **Ergebnisanalyse**: Intelligente Interpretation der Daten
6. **Benutzeroberfläche**: Gradio-basiertes Chatinterface für einfache Interaktion

Die Anwendung zeigt den vollständigen Workflow von SAG:

1. Der Benutzer stellt eine Frage in natürlicher Sprache
2. Das LLM generiert eine passende SQL-Abfrage
3. Die Abfrage wird ausgeführt und die Ergebnisse formatiert
4. Ein zweiter LLM-Aufruf analysiert und interpretiert die Ergebnisse
5. Die formatierte Antwort wird dem Benutzer präsentiert

Diese Implementierung demonstriert, wie SAG komplexe Datenanalysen für Benutzer ohne SQL-Kenntnisse zugänglich macht und gleichzeitig präzise, datenbasierte Antworten liefert.



**Datenbank-Schema:**

![Northwind E-R Diagramm](https://upload.wikimedia.org/wikiversity/en/a/ac/Northwind_E-R_Diagram.png)


[Quelle:](https://upload.wikimedia.org/wikiversity/en/a/ac/Northwind_E-R_Diagram.png)

<p><font color='black' size="5">
Installation und API-Keys
</font></p>

In [ ]:
# Abschnitt 0: Installation und API-Key
%%writefile requirements.txt
langchain>=0.2
langchain-experimental>=0.0.49
langchain-openai>=0.0.5
openai>=1.55.3
httpx>=0.27.2
sqlalchemy>=2.0.0
gradio>=4.0.0
pydantic>=2.0.0
python-dotenv>=1.0.0

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install -q -U --system -r requirements.txt

<p><font color='black' size="5">
Datenbank kopieren
</font></p>

In [ ]:
# Northwind-Datenbank herunterladen
!rm -rf northwind.db
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/northwind.db -o northwind.db

<p><font color='black' size="5">
Programm
</font></p>

In [ ]:
# Standard & Third Party Libraries
import sqlite3
import re
import gradio as gr
from langchain_experimental.sql.base import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

DB_PATH = "/content/northwind.db"
DB_URI = f"sqlite:///{DB_PATH}"

# LLM initialisieren
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# SQL-Datenbank initialisieren
db = SQLDatabase.from_uri(DB_URI)

# Erweiterten Prompt für SQL-Abfragen erstellen
template = """
Du bist ein SQL-Experte. Deine Aufgabe ist es, Benutzeranfragen in SQL-Abfragen zu übersetzen.
Verwende die SQLite-Syntax und nur die Tabellen und Spalten aus dem bereitgestellten Schema.
Schreibe NUR die SQL-Abfrage ohne Präfixe oder Kommentare.
Gebe neben den Id auch den Namen von Produkten, Kunden, etc. mit aus.
Gebe maximal 10 Zeilen einer Liste aus.

Wichtig: Bei Ja/Nein-Fragen oder Fragen, die eine Analyse erfordern (z.B. "Sind alle Artikel auf Lager?"),
erstelle eine SQL-Abfrage, die ALLE relevanten Daten zurückgibt, damit eine fundierte Antwort gegeben werden kann.
Für komplexe Fragen mit Bedingungen wie "vom 1998-05-06" oder einem bestimmten Kundennamen,
stelle sicher, dass diese Bedingungen in der WHERE-Klausel korrekt berücksichtigt werden.
Achte darauf, ob bei der Frage nach einer Id oder dem Namen von Produkten, Kunden, Unternehmen, etc. gefragt wird.

Datenbank-Schema:
{schema}

Benutzeranfrage: {query}

SQL-Abfrage:
"""

# Template für die Ergebnisinterpretation
analysis_template = """
Du bist ein Business-Analyst, der SQL-Abfrageergebnisse interpretiert und verständliche Antworten gibt.
Beantworte die Benutzeranfrage basierend auf den SQL-Ergebnissen.

Bei Ja/Nein-Fragen gib eine klare Antwort und erkläre die Gründe.
Bei Fragen nach Empfehlungen oder notwendigen Anpassungen, analysiere die Daten und gib konkrete Vorschläge.

Benutzeranfrage: {query}
SQL-Abfrage: {sql_query}
Abfrageergebnisse:
{results}

Deine Analyse und Antwort:
"""

def get_schema(_):
    return db.get_table_info()

sql_generator = (
    RunnablePassthrough.assign(schema=get_schema)
    | PromptTemplate.from_template(template)
    | llm
    | StrOutputParser()
)

def execute_query(sql_query: str) -> str:
    """Führt eine SQL-Abfrage aus und formatiert die Ergebnisse als String."""
    try:
        # Bereinige die Abfrage von eventuellen Formatierungen
        cleaned_query = sql_query.strip()

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute(cleaned_query)

        # Spaltenüberschriften abrufen
        column_names = [description[0] for description in cursor.description]

        # Ergebnisse abrufen
        results = cursor.fetchall()

        # Ergebnisse formatieren
        output = "| " + " | ".join(column_names) + " |\n"
        output += "| " + " | ".join(["---" for _ in column_names]) + " |\n"

        for row in results:
            output += "| " + " | ".join([str(cell) for cell in row]) + " |\n"

        conn.close()

        # Keine Ergebnisse gefunden
        if len(results) == 0:
            return "Keine Ergebnisse gefunden."

        return output

    except Exception as e:
        return f"Fehler bei der Ausführung der Abfrage: {str(e)}\nAbfrage: {cleaned_query}"

def analyze_results(query, sql_query, results):
    """Analysiert die Ergebnisse und gibt eine natürlichsprachliche Antwort zurück."""
    analysis_prompt = PromptTemplate.from_template(analysis_template)
    analysis_chain = analysis_prompt | llm | StrOutputParser()

    return analysis_chain.invoke({
        "query": query,
        "sql_query": sql_query,
        "results": results
    })

def chatbot_response(message, history):
    """Verarbeitet Benutzeranfragen, erstellt SQL und gibt formatierte Ergebnisse mit Analyse zurück."""
    try:
        # SQL-Abfrage mit LLM generieren
        sql_query = sql_generator.invoke({"query": message})

        # Bereinige eventuelles Markdown-Markup
        sql_query = re.sub(r'```sql\s*(.*?)\s*```', r'\1', sql_query, flags=re.DOTALL)
        sql_query = sql_query.replace("```", "").strip()

        # Debug-Ausgabe
        print(f"Generierte SQL: {sql_query}")

        # Führe die Abfrage aus
        results = execute_query(sql_query)

        # Analysiere die Ergebnisse für komplexe Fragen
        analysis = analyze_results(message, sql_query, results)

        # Antwort formatieren
        response = f"### Deine Anfrage\n{message}\n\n### SQL-Abfrage\n```sql\n{sql_query}\n```\n\n### Ergebnisse\n{results}\n\n### Analyse\n{analysis}"

        return response

    except Exception as e:
        return f"Ein Fehler ist aufgetreten: {str(e)}"

# Beispielfragen definieren
example_questions = [
    "Welche Produkte sind aktuell nicht mehr auf Lager? Nenne die Top 3.",
    "Welche Bestellung von welchem Kunden hatte den höchsten Gesamtwert? Nenne die Top 3.",
    "Aus welchen Ländern stammen die meisten Kunden? Nenne die Top 3.",
    "Sind alle Artikel der Bestellung der Rattlesnake Canyon Grocery vom 1998-05-06 in ausreichender Anzahl auf Lager?"
]

# Gradio Interface erstellen
demo = gr.ChatInterface(
    fn=chatbot_response,
    title="📚 Erweiterte SQL-Augmented Generation (SAG)",
    description="\n\n*Der Chatbot wertet die Datenbank aus, beantwortet Fragen zum Inhalt und gibt Handlungsempfehlungen*",
    examples=example_questions,
    type="messages"  # Aktualisiert auf das neue OpenAI-Style Nachrichtenformat
)

<p><font color='black' size="5">
Starten der App
</font></p>

**Beispiel-Fragen:**



+ Gib die Artikelliste für die Bestellung 11031 mit Einzelpreis und Gesamtpreis aus, wobei sich der Gesamtpreis aus der Anzahl und dem Einzelpreis ergibt.
+ Welcher Mitarbeiter ist für die Bestellung mit der Nummer 10266 zuständig?
+ Über welche Versandfirma wurde die Bestellung 10266 ausgeliefert?
+ Sind alle Artikel der Bestellung der Rattlesnake Canyon Grocery vom 1998-05-06 in ausreichender Anzahl auf Lager?
+ Welche Kunden haben schon Artikel der Firma 'Escargots Nouveaux' gekauft?




In [ ]:
# App starten
demo.launch()

# **9 <font color='orange'>|</font> Zusammenfassung**
---



SQL-Augmented Generation (SAG) bietet einen leistungsstarken Ansatz, um LLMs mit strukturierten Datenbanken zu verbinden. Durch die Kombination von natürlicher Sprachverarbeitung mit präzisen Datenbankabfragen ermöglicht SAG:

- Präzise, faktenbasierte Antworten auf komplexe Fragen
- Zugang zu aktuellen Daten für Geschäftsanalysen
- Demokratisierung des Datenzugriffs für Nicht-Techniker
- Automatisierte Berichterstellung und Datenanalyse

Mit Frameworks wie LangChain ist die Implementierung von SAG-Lösungen zugänglicher geworden, was neue Möglichkeiten für datengesteuerte Anwendungen eröffnet.

# **A <font color='orange'>|</font> Aufgabe**
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

Angenommen, es wird für ein kleines Unternehmen gearbeitet, das eine Kundendatenbank verwaltet. Ziel ist es, eine generative KI einzusetzen, um Anfragen in natürlicher Sprache zu verstehen und relevante Informationen aus der Datenbank abzurufen.  

**Datenbankstruktur (SQLite-Format)**  
Die Kundendatenbank enthält eine Tabelle `customers.db` mit den folgenden Spalten:  

| id | name  | city    | purchases |
|----|-------|--------|-----------|
| 1  | Alice  | Berlin  | 5         |
| 2  | Bob    | Hamburg | 2         |
| 3  | Carol  | München | 7         |
| 4  | David  | Köln    | 3         |



**Aufgabenstellung**  
1. **Abfrage erstellen**, um die Anzahl der Einkäufe (`purchases`) eines bestimmten Kunden anhand seines Namens abzurufen.  
2. **Python-Funktion entwickeln**, die eine GPT-API nutzt, um natürliche Sprachabfragen in SQL-Abfragen zu übersetzen.  
3. **Funktion testen**, indem eine Frage wie *„Wie viele Einkäufe hat Alice gemacht?“* gestellt wird, woraufhin das System automatisch die entsprechende SQL-Abfrage generiert.  

